# HVNM Large-Amplitude Oscillatory Shear (LAOS): Payne Effect and Harmonic Generation

## Introduction

In LAOS ($\gamma(t) = \gamma_0 \sin(\omega t)$), the HVNM exhibits **dual TST nonlinearity** from both matrix (E-network) and interphase (I-network) stress-dependent bond exchange kinetics. The **Payne effect** emerges naturally: strain amplification $X_I$ reduces the effective critical strain to $\gamma_c^{eff} = \gamma_c / X_I$, causing earlier onset of nonlinearity at higher $\phi$. The stress response contains odd harmonics, with $I_3/I_1$ quantifying nonlinearity. Lissajous curves transition from ellipses (linear) to distorted loops (nonlinear) as $\gamma_0$ or $\phi$ increases.

> **Handbook:** See [HVNM Protocol Derivations](../../docs/source/models/hvnm/hvnm_protocols.rst) for LAOS ODE formulation and [Knowledge Extraction](../../docs/source/models/hvnm/hvnm_knowledge.rst) for Payne onset analysis.

## Learning Objectives

- Visualize elastic and viscous Lissajous curves with NP amplification
- Understand dual TST harmonic generation (E-network + I-network)
- Observe Payne effect: strain amplification lowers critical strain $\gamma_c$
- Analyze amplitude-dependent nonlinearity with odd harmonics

## Prerequisites
- **Notebook 01** — SAOS fundamentals
- **Notebook 03** — Startup shear and TST kinetics

## Estimated Runtime
- ~3 min (17-component ODE with oscillatory forcing)

In [1]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

Running locally


In [2]:
# Imports
from rheojax.core.jax_config import safe_import_jax, verify_float64

jax, jnp = safe_import_jax()
verify_float64()

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.models import HVNMLocal

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
import os
import sys

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

JAX version: 0.8.3
Devices: [CpuDevice(id=0)]


## Theory: LAOS and Dual TST Nonlinearity

In large-amplitude oscillatory shear (LAOS), the strain is:

$$\gamma(t) = \gamma_0 \sin(\omega t)$$

For the HVNM model, nonlinearity arises from:

1. **Dual TST stress dependence**: Both E-network and I-network bond exchange rates depend on stress via k_ex(σ) ∼ exp(V_act·σ/(RT))
2. **Strain amplification**: Nanoparticles amplify local strain → earlier onset of nonlinearity
3. **Natural state evolution**: Reference configurations evolve nonlinearly with deformation
4. **Payne effect**: Critical strain γ_c reduced by factor X_I due to amplification

The stress response contains odd harmonics:

$$\sigma(t) = \sum_{n=1,3,5,...} [G'_n \sin(n\omega t) + G''_n \cos(n\omega t)]$$

The third harmonic intensity I₃/I₁ quantifies nonlinearity. For small amplitudes, I₃/I₁ → 0 and the response becomes linear.

**Payne Effect**: The effective critical strain is γ_c_eff = γ_c / X_I. Higher nanoparticle loading (φ) → larger X_I → earlier onset of nonlinearity.

**Lissajous curves** plot:
- **Elastic**: σ vs γ (ellipse → distorted loop)
- **Viscous**: σ vs γ̇ (ellipse → distorted loop)

In [3]:
# Model setup with strong TST coupling
model = HVNMLocal(kinetics="stress")
model.parameters.set_value("G_P", 5000.0)    # Permanent network modulus [Pa]
model.parameters.set_value("G_E", 3000.0)    # E-network (exchangeable) [Pa]
model.parameters.set_value("G_D", 1000.0)    # D-network (dangling) [Pa]
model.parameters.set_value("phi", 0.1)       # Nanoparticle volume fraction
model.parameters.set_value("beta_I", 3.0)    # Immobilized fraction (I-network)
model.parameters.set_value("T", 350.0)       # Temperature [K]
model.parameters.set_value("k_d_D", 10.0)    # D-network dissociation rate [1/s]
model.parameters.set_value("nu_0", 1e10)     # TST attempt frequency [1/s]
model.parameters.set_value("E_a", 80e3)      # Activation energy [J/mol]
model.parameters.set_value("V_act", 1e-4)    # Activation volume - INCREASED for TST

phi = model.phi
X_I = 1.0 + 2.5*phi + 14.1*phi**2  # Guth-Gold

print("HVNM Model Parameters:")
print(f"  G_P = {model.G_P:.0f} Pa")
print(f"  G_E = {model.G_E:.0f} Pa")
print(f"  G_D = {model.G_D:.0f} Pa")
print(f"  φ = {model.phi:.2f}")
print(f"  β_I = {model.beta_I:.1f}")
print(f"  X_I(φ={phi:.2f}) = {X_I:.3f} (amplification factor)")
print(f"  T = {model.T:.0f} K")
print(f"  V_act = {model.V_act:.2e} m³/mol (strong TST coupling)")


HVNM Model Parameters:
  G_P = 5000 Pa
  G_E = 3000 Pa
  G_D = 1000 Pa
  φ = 0.10
  β_I = 3.0
  X_I(φ=0.10) = 1.391 (amplification factor)
  T = 350 K
  V_act = 1.00e-04 m³/mol (strong TST coupling)


In [4]:
# LAOS simulation
omega = 1.0
gamma_0 = 0.5
n_cycles = 10
t = np.linspace(0, n_cycles * 2 * np.pi / omega, n_cycles * 200)

print(f"Simulating LAOS: γ₀ = {gamma_0}, ω = {omega:.1f} rad/s, {n_cycles} cycles...")
result = model.simulate_laos(t, gamma_0=gamma_0, omega=omega)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Time series
n_per = int(len(t) / n_cycles)
t_last = t[-n_per:]
ax1.plot(t_last, result["strain"][-n_per:], "b-", lw=1.5, label="Strain γ")
ax1_twin = ax1.twinx()
ax1_twin.plot(t_last, result["stress"][-n_per:], "r-", lw=1.5, label="Stress σ")
ax1.set_xlabel("t [s]")
ax1.set_ylabel("Strain γ", color="b")
ax1_twin.set_ylabel("Stress σ [Pa]", color="r")
ax1.set_title("LAOS Time Series (last cycle)")
ax1.grid(True, alpha=0.3)

# Elastic Lissajous (σ vs γ)
ax2.plot(result["strain"][-n_per:], result["stress"][-n_per:], "k-", lw=2)
ax2.set_xlabel("Strain γ")
ax2.set_ylabel("Stress σ [Pa]")
ax2.set_title("Elastic Lissajous Curve (last cycle)")
ax2.grid(True, alpha=0.3)
ax2.axhline(0, color="k", lw=0.5, alpha=0.5)
ax2.axvline(0, color="k", lw=0.5, alpha=0.5)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nStress range: {result['stress'].min():.2f} to {result['stress'].max():.2f} Pa")

Simulating LAOS: γ₀ = 0.5, ω = 1.0 rad/s, 10 cycles...


<Figure size 1400x500 with 3 Axes>


Stress range: -7538.14 to 7494.89 Pa


In [5]:
# Viscous Lissajous and harmonic extraction
gamma_dot_t = gamma_0 * omega * np.cos(omega * t)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Viscous Lissajous (σ vs γ̇)
ax1.plot(gamma_dot_t[-n_per:], result["stress"][-n_per:], "r-", lw=2)
ax1.set_xlabel("Strain rate γ̇ [1/s]")
ax1.set_ylabel("Stress σ [Pa]")
ax1.set_title("Viscous Lissajous Curve (last cycle)")
ax1.grid(True, alpha=0.3)
ax1.axhline(0, color="k", lw=0.5, alpha=0.5)
ax1.axvline(0, color="k", lw=0.5, alpha=0.5)

# Harmonic extraction
print("\nExtracting Fourier harmonics from LAOS stress signal...")
harmonics_raw = model.extract_laos_harmonics(result, n_harmonics=5)
harm_idx = harmonics_raw["harmonic_index"]
harm_amp = harmonics_raw["sigma_harmonics"]

if len(harm_idx) > 0:
    ax2.bar(harm_idx, harm_amp, color='steelblue', edgecolor='black', alpha=0.7)
    ax2.set_xlabel("Harmonic n")
    ax2.set_ylabel("|I_n| [Pa]")
    ax2.set_title("Harmonic Intensities")
    ax2.set_xticks(harm_idx)
    ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
display(fig)
plt.close(fig)

print("\nFourier Harmonics (stress):")
print(f"{'n':>3} {'|I_n|':>12} {'I_n/I_1':>12}")
print("-" * 30)
I1 = harm_amp[0] if len(harm_amp) > 0 else 1.0
for n, amp in zip(harm_idx, harm_amp):
    ratio = amp / max(I1, 1e-30)
    print(f"{n:>3} {amp:>12.2f} {ratio:>12.4f}")

if len(harm_amp) >= 2 and I1 > 0:
    nonlinearity_ratio = harm_amp[1] / I1  # I3/I1
    print(f"\nNonlinearity ratio I₃/I₁ = {nonlinearity_ratio:.4f}")
    if nonlinearity_ratio < 0.01:
        print("→ Nearly linear response (I₃/I₁ < 1%)")
    elif nonlinearity_ratio < 0.1:
        print("→ Weakly nonlinear response (1% < I₃/I₁ < 10%)")
    else:
        print("→ Strongly nonlinear response (I₃/I₁ > 10%)")


Extracting Fourier harmonics from LAOS stress signal...


<Figure size 1400x500 with 2 Axes>


Fourier Harmonics (stress):
  n        |I_n|      I_n/I_1
------------------------------
  1      7503.00       1.0000
  3         3.15       0.0004
  5         1.82       0.0002
  7         1.31       0.0002
  9         1.04       0.0001

Nonlinearity ratio I₃/I₁ = 0.0004
→ Nearly linear response (I₃/I₁ < 1%)


In [6]:
# Payne effect: amplitude sweep with φ comparison
amplitudes = [0.01, 0.1, 0.5, 1.0]
phi_values = [0.0, 0.1, 0.2]
colors_phi = ['blue', 'green', 'red']

print("\nPayne effect demonstration: φ comparison at different γ₀...")
fig, axes = plt.subplots(1, len(amplitudes), figsize=(4*len(amplitudes), 4))

for ax, g0 in zip(axes, amplitudes):
    for phi_val, color in zip(phi_values, colors_phi):
        model_temp = HVNMLocal(kinetics="stress")
        model_temp.parameters.set_value("G_P", 5000.0)
        model_temp.parameters.set_value("G_E", 3000.0)
        model_temp.parameters.set_value("G_D", 1000.0)
        model_temp.parameters.set_value("phi", phi_val)
        model_temp.parameters.set_value("beta_I", 3.0)
        model_temp.parameters.set_value("T", 350.0)
        model_temp.parameters.set_value("k_d_D", 10.0)
        model_temp.parameters.set_value("nu_0", 1e10)
        model_temp.parameters.set_value("E_a", 80e3)
        model_temp.parameters.set_value("V_act", 1e-4)
        
        r = model_temp.simulate_laos(t, gamma_0=g0, omega=omega)
        X = 1.0 + 2.5*phi_val + 14.1*phi_val**2
        n_per_temp = int(len(t) / n_cycles)
        ax.plot(r["strain"][-n_per_temp:], r["stress"][-n_per_temp:], 
                "-", lw=1.5, color=color, label=f"φ={phi_val:.1f}")
    
    ax.set_xlabel("γ")
    ax.set_title(f"γ₀ = {g0}")
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color="k", lw=0.5, alpha=0.3)
    ax.axvline(0, color="k", lw=0.5, alpha=0.3)
    if ax == axes[0]:
        ax.legend()

axes[0].set_ylabel("σ [Pa]")
plt.suptitle("Payne Effect: Higher φ → Earlier Nonlinearity Onset", fontsize=12, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

print("\n→ Observe: Higher φ causes stronger Lissajous distortion at same γ₀")
print("→ Reason: Strain amplification X_I lowers effective critical strain γ_c/X_I")


Payne effect demonstration: φ comparison at different γ₀...


<Figure size 1600x400 with 4 Axes>


→ Observe: Higher φ causes stronger Lissajous distortion at same γ₀
→ Reason: Strain amplification X_I lowers effective critical strain γ_c/X_I


## Key Takeaways

1. **Dual TST nonlinearity**: Both E-network and I-network contribute odd harmonics via stress-dependent $k_{ex}$
2. **Payne effect**: Strain amplification $X_I$ reduces critical strain → earlier onset of nonlinearity
3. **$I_3/I_1$ quantifies nonlinearity**: Third harmonic ratio increases with amplitude $\gamma_0$ and $\phi$
4. **$\phi$-dependent transient**: Unlike steady flow ($\phi$-independent), LAOS shows strong $\phi$ effects
5. **Lissajous distortion**: Transition from ellipse (linear) to asymmetric loops (nonlinear)

## Further Reading

**Handbook Documentation:**
- [HVNM Protocol Derivations](../../docs/source/models/hvnm/hvnm_protocols.rst) — LAOS ODE formulation, harmonic extraction
- [HVNM Knowledge Extraction](../../docs/source/models/hvnm/hvnm_knowledge.rst) — Payne onset strain $\gamma_c / X_I$, TST validation

**Key References:**
1. Karim, M.R., Vernerey, F. & Sain, T. (2025). *Macromolecules*, 58(10), 4899-4912.
2. Li, Z., et al. (2024). "Manipulating Vitrimer Nanocomposite Properties." *Langmuir*, 40(14), 7550-7560.

## Next Notebooks

- **Notebook 07**: Limiting cases — Factory methods and $\phi=0$ HVM recovery
- **Notebook 08**: Data intake and QC workflows
- **Notebook 14**: LAOS NLSQ/NUTS fitting